 ## pandas 설정

In [1]:
import pandas as pd
print("pandas version: ", pd.__version__)
#pd.set_option('display.max_row', 1000000)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000000)
pd.set_option('display.max_seq_items', None)
#pd.set_option('display.height', 1000)
pd.set_option('display.max_colwidth', -1)

pandas version:  1.4.1


C:\Users\karas\AppData\Local\Temp/ipykernel_13728/2241320340.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [1]:
#jupyter 넓이 늘리기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

C:\Users\karas\AppData\Local\Temp\ipykernel_5412\2126364666.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
!pip install numpy

     ---------------------------------------- 14.6/14.6 MB 3.7 MB/s eta 0:00:00


In [6]:
# import struct
import re
import os
import csv
import datetime
import requests
import numpy as np
from mmap import ACCESS_READ, mmap

now = datetime.datetime.now()
nowDate = now.strftime('%Y-%m-%d')
#nowDate = now.strftime('%Y-%m-%d_%H_%M_%S')
path = input("write full path: ")
try:
    handle = open(path, 'rb+')
except IOError:
    print('IOError : File path is invalid')
    sys.exit()
print(path)

def showmenu():  # 메뉴 선택 함수
    print()
    print("[Ethereum Aritifacts search choose menu]")
    for i in range(0, len(menu)):
        print(i+1, ".", menu[i])
    print()
    
def artifacts():
    if sel == 1:
        p = re.compile(rb'^0x[a-fA-F0-9]{40}', re.IGNORECASE | re.MULTILINE) #이더리움 주소, 0x[0-9A-Fa-f]{40,44}
    elif sel == 2:
        p = re.compile(rb'[0-9a-f]{64}', re.IGNORECASE | re.MULTILINE) #이더리움 개인키
    elif sel == 3:
        p = re.compile(rb'"([address"]*)":"([a-fA-F0-9]{40})","([crypto"]*)":', re.IGNORECASE | re.MULTILINE) #이더리움 키스토어
    elif sel == 4:
        p = re.compile(rb'[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}', re.MULTILINE) # 나모닉 코드 12자리
    elif sel == 5:
        p = re.compile(rb'[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}\s[a-z]{3,8}', re.IGNORECASE | re.MULTILINE) # 나모닉 코드 24자리
    else:
        print("another artifacts?")
   
    print("============ Next Step ============")

    with open(path, 'rb') as file, mmap(file.fileno(), length=0, access=ACCESS_READ) as mm:
        b=type(mm)
        #print(b)
        for match in p.finditer(mm):
            s = match.start()
            e = match.end()
            grep_match_found = mm[s:e].decode("utf-8")
            match_result = str(grep_match_found) #결과값 String
            with open('c:\\temp\\'+f'{nowDate}'+'_match_1차_list.txt', '+a') as file:
                if sel == 1:
                    file.write(match_result+'\n')
                elif sel == 4 and len(match_result.split()) == 12:
                    print("1차 검색 완료", match_result)
                    file.write(match_result+'\n')
                file.close()
        #mm.close()
            #여기서 부터 address 한줄 씩 받아서 검증 및 중복값 제거
            if sel == 1:
                apikey = "WP3JPWBB21D6JRN1AVIE451N4H5YVTJN4M" 
                URL = f"https://api.etherscan.io/api?module=account&action=txlist&apikey={apikey}&sort=desc&address={match_result}"
                if match_result in URL:
                    global csv_file
                    csv_file = 'c:\\temp\\'+f'{nowDate}'+'_ethereum_address_scan_result.csv'
                    headers = {'User-Agent': 'Mozilla/5.0'} # header 선언, http 요청 브라우저는 Mozilla.
                    resp = requests.get(URL).json() # http get 요청을 resp 변수에 json 형태로 선언
                    for key, value in resp.items():  # apikey error test
                        if value == "1":
                            print(URL)
                            global api_result, ts_result, to_result, hx_result
                            status = resp["message"]
                            data = resp["result"] # result 결과인 value만 변수에 담기
                            api_result = dict(data[0])
                            ts_result = api_result.get('timeStamp')
                            to_result = api_result.get('to')
                            hx_result = api_result.get('hash')
                            csv_print()
                            break
                break
            elif sel == 2:
                print("sel 2")
            elif sel == 3:
                print(match_result)
            elif sel == 4:               
                with open("G:\\내 드라이브\\04 - Ethereum_study\\ethereum_forensic\\mnemonic_word.txt",'r', encoding='UTF8') as file:
                    text = file.read() # 니모닉 코드 단어 사전 읽기
                    #mnemonic_file = open('c:\\temp\\'+f'{nowDate}'+'_match_1차_list.txt','r')
                    #line = mnemonic_file.readline() # 니모닉 코드 1차 검색 결과 한줄씩
                    if match_result in text:
                        print("a")
                        #print(line) # 여기까지 수행.2022.09.17 새벽 2시
                        #if line in text:
                        flag = True
                        print(line)
                        with open('c:\\temp\\match_2차_list.txt', '+a') as file_second:
                            file_second.write(line+'\n')
                            file_second.close()
                    else:
                        flag = False
                        #print("not matching")
                    #line = mnemonic_file.readline()
                    #mnemonic_file.close()                              
        file.close()
        mm.close()
        
def csv_print():
    with open(csv_file, '+a', encoding='utf-8-sig') as csvfile:
        #writer = csv.writer(csvfile, fieldnames=['timeStamp', 'to', 'hash', 'isError', 'gas', 'methodId', 'blockNumber', 'cumulativeGasUsed', 'blockHash', 'gasPrice', 'contractAddress', 'nonce', 'transactionIndex', 'confirmations', 'value', 'from', 'functionName', 'input', 'txreceipt_status', 'gasUsed'])
        writer = csv.writer(csvfile)
        #writer.writeheader()
        if sel == 1:
            writer.writerow([ts_result, to_result, hx_result])
        elif sel == 4:
            writer.writerow(match_mnemonic)

def mnemonic_word():
    global MNEM_LIST
    dicfile = open("G:\\내 드라이브\\04 - Ethereum_study\\ethereum_forensic\\mnemonic_word.csv",'r')
    MNEMONIC_DIC = csv.reader(dicfile)
    MNEM_LIST = list(MNEMONIC_DIC)
    dicfile.close()
    
if __name__ == '__main__':
    menu = ("address", "private key", "keystore", "mnemonic code(12)", "mnemonic code(24)")
    while True:
        showmenu()
        sel = int(input("Choose Menu number (Exit 0) : "))
        if sel == 0:
            break
        elif(sel>=1 and sel <= len(menu)):
            #transactions("")
            artifacts()
            print ("transaction result print")
            sel1 = str(input("[*] Do you want to continue searching? if you enter 'no', quit process : "))
            if sel1 == "no":
                break
            elif sel1 == "yes":
                print ("keep going")
                artifacts()
            else : 
                print ("quit")
        else:
            print("잘못된 메뉴 번호입니다.")
    print ("complete search")

#0x9ae08d0118e105dce648a7fba6fc12b2c3e9c288
# G:\내 드라이브\mem_dump\memdump.mem
# G:\내 드라이브\04 - Ethereum_study\ethereum_tracking\051021-8734-01.dmp

write full path: G:\내 드라이브\mem_dump\memdump.mem
G:\내 드라이브\mem_dump\memdump.mem

[Ethereum Aritifacts search choose menu]
1 . address
2 . private key
3 . keystore
4 . mnemonic code(12)
5 . mnemonic code(24)

Choose Menu number (Exit 0) : 4
============ Next Step ============
1차 검색 완료 emory was accessed during this time that was not properly marked
for
1차 검색 완료 kipping file copy because source and target full path are the same
1차 검색 완료 nodev	sysfs
nodev	rootfs
nodev	bdev
nodev	proc
nodev	tmpfs
nodev	binfmt
1차 검색 완료 asily create visual and engaging word clouds from data pulled straight from
1차 검색 완료 apology also recycle argue dirt bring silver mix rebuild minute era ivory
1차 검색 완료 jan	feb	mar	apr	may	jun	jul	aug	sep	oct	nov	dec
1차 검색 완료 laundry
prize
hospital
run
hammer
treat
lemon
honey
boss
blanket
pave
spider
1차 검색 완료 readers should also detect the case where data blocks are missing from
1차 검색 완료 the raw image data has already had its black encoding level subtract
1차 검색 완료 about
actual

1차 검색 완료 organize the present and prepare for the future with our visual and
1차 검색 완료 secure vendors and partners for current and future projects and help bidding
1차 검색 완료 atch constant function must use the same input control point type declared
1차 검색 완료 dilemma road print laugh follow open indoor swing scorpion sugar planet whale
1차 검색 완료 ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff ffffffff
1차 검색 완료 apology also recycle argue dirt bring silver mix rebuild minute era ivory
1차 검색 완료 question you need answered and letting people can add their bulleted ideas
1차 검색 완료 oxygen scrap buddy great account wrap slight chaos farm brain real chest
1차 검색 완료 nmuting failed and the element was paused instead because the user didn
1차 검색 완료 the friendly data tool for modern teams who work with numbers and
1차 검색 완료 enerator prefix
link and archive commands with the given launcher command line
1차 검색 완료 elect tracker became dormant after firing drag 

1차 검색 완료 may create groups and items only for boards you created and assigned
1차 검색 완료 rivi rivipv rivzd vsr rldoc rlfi rlfl rlhd rlhs rlimm rlmf rlvp
1차 검색 완료 rlmlel rltbs rpd rrid rsnr rsrs rspi sabec sabptc sabs sabsd sbfbu
1차 검색 완료 siv sie scso scrl slo schid smids smr smrq sng snsb spd
1차 검색 완료 twd twmlbx vet ugc piv ugcqalb umvtfk umvreg umvver vch view viewer
1차 검색 완료 apology also recycle argue dirt bring silver mix rebuild minute era ivory
1차 검색 완료 the proxy target but the proxy did not return its actual value
1차 검색 완료 fenced frame root whose owner frame has the same fenced frame nonce
1차 검색 완료 fenced frame root whose owner frame has the same fenced frame nonce
1차 검색 완료 rive terus daripada penyemak imbas anda dalam aplikasi serasi yang dipasang pada
1차 검색 완료 question you need answered and letting people can add their bulleted ideas
1차 검색 완료 losed captions text track display name format that includes the language and
1차 검색 완료 insert well known storage root into assembly storage 

1차 검색 완료 inm ion ircip isn kapk lei lrad lsft luul mapsl muul mvs
1차 검색 완료 sclient scsr sert sesinv site sla sns source sourceid spell spknlang sqi
1차 검색 완료 ota asal dan sebarang nota yang dibuat daripada templat ini tidak akan
1차 검색 완료 share feedback slots when loading the same named property from the same
1차 검색 완료 fel uppstod och det gick inte att kryptera och ladda upp dina
1차 검색 완료 apology also recycle argue dirt bring silver mix rebuild minute era ivory
1차 검색 완료 nda akan melihat versi lite dasbor sehingga aplikasi dapat dibuka lebih cepat
1차 검색 완료 second second seconds seconds seconds seconds seconds sec nds sec nds setting
1차 검색 완료 fenced frame root whose owner frame has the same fenced frame nonce
1차 검색 완료 his plugin does not support font engines created directly from font data
1차 검색 완료 how any events that are upcoming and all events that have already
1차 검색 완료 comment for feedback for other faculty members that share the same students
1차 검색 완료 because the user changed the passwo

1차 검색 완료 muffin canal chimney glimpse target fold setup card flee journey distance argue
1차 검색 완료 read the upload data stream after the stream has already been read
1차 검색 완료 terate over all stack frames and look for the first one not
1차 검색 완료 laundry
prize
hospital
run
hammer
treat
lemon
honey
boss
blanket
pave
spider
1차 검색 완료 server could not process the resource and returned this brief error message
1차 검색 완료 the runtime notices this and returns the already created object for that
1차 검색 완료 the runtime notices this and returns the already created object for that
1차 검색 완료 these keywords identify the setting that applies the last text color formatti
1차 검색 완료 oxygen scrap buddy great account wrap slight chaos farm brain real chest
1차 검색 완료 oxygen scrap buddy great account wrap slight chaos farm brain real chestide
1차 검색 완료 laundry
prize
hospital
run
hammer
treat
lemon
honey
boss
blanket
pave
spider
1차 검색 완료 oxygen scrap buddy great account wrap slight chaos farm brain real chest
1차 검색 완료 

In [49]:
#def mnemonic_word():
#    global MNEM_LIST
dicfile = open("G:\\내 드라이브\\04 - Ethereum_study\\ethereum_forensic\\mnemonic_word.txt",'r')
MNEMONIC_DIC = csv.reader(dicfile)
MNEM_LIST = list(MNEMONIC_DIC)
word = input("input word: ")
if word in MNEM_LIST:
    print(word)
else:
    print("don't exist")
#print(MNEM_LIST[1])
dicfile.close()
'''
with open("G:\\내 드라이브\\04 - Ethereum_study\\ethereum_forensic\\mnemonic_word.txt",'r', encoding='UTF8') as file:
    text = file.read()
    print(type(text))
    print(text)
'''


input word: are
don't exist


'\nwith open("G:\\내 드라이브\\04 - Ethereum_study\\ethereum_forensic\\mnemonic_word.txt",\'r\', encoding=\'UTF8\') as file:\n    text = file.read()\n    print(type(text))\n    print(text)\n'

In [2]:
mnemonic_file = open('c:\\temp\\'+f'{nowDate}'+'_match_1차_list.txt','r')
line_num = 1
line = mnemonic_file.readline()
while line:
    #print(type(line))
    print(line)
    line = mnemonic_file.readline()

NameError: name 'nowDate' is not defined

### 메모리 덤프에서 csv로 추출된 이더리움 주소 정규표현식에 해당하는 데이터 개수

In [3]:
# csv 행 수 출력하기
# https://why2why.tistory.com/116

import pandas as pd
import os

fpath = 'F:\\ethereum_forensic\\'
fall = os.listdir(fpath)
fcsv = [fpath + file[:-4] for file in fall if file.endswith('ethereum_address_scan.csv')]
#fcsv = 'ethereum_address_scan.csv'
print(fcsv)

for file in fcsv:
    df = pd.read_csv(file + '.csv')
    print(file, len(df))
    


['F:\\ethereum_forensic\\ethereum_address_scan']
F:\ethereum_forensic\ethereum_address_scan 111758


In [1]:
'''import csv

datafile = open('F:\\ethereum_forensic\\ethereum_address_scan.csv','r')

datafile_read = csv.reader(datafile)
for line in datafile_read:
    print(line)

datafile.close()
'''

"import csv\n\ndatafile = open('F:\\ethereum_forensic\\ethereum_address_scan.csv','r')\n\ndatafile_read = csv.reader(datafile)\nfor line in datafile_read:\n    print(line)\n\ndatafile.close()\n"

### 주소/개인키/키스토어/니모닉코드 추출 -> 학회 논문
### 주소/개인키/키스토어/니모닉코드 추출 및 웹 UI 구현, 시각화 표현 -> 학위논문 (다크웹 포함)


In [4]:
!pip install mnemonic

     ---------------------------------------- 62.0/62.0 kB ? eta 0:00:00


In [3]:
#!pip install web3
from web3.auto import w3

w3.eth.account.enable_unaudited_hdwallet_features()
# 니모닉 코드 입력하여 주소 확인
account = w3.eth.account.from_mnemonic("laundry prize hospital run hammer treat lemon honey boss blanket pave spider")
# 개인키 입력하여 주소 확인
#account = w3.eth.account.privateKeyToAccount("247c8b5dc83ef1093a6e43ba75a334c74775e61496a445d715a5119fee592618")

print(account.address)



0xd8a0734DDB3cF797dC91E80a97D39a6Cee3eF6c6


In [6]:
from eth_account import Account
import secrets
priv = secrets.token_hex(32)
private_key = "0x" + priv
print("SAVE BUT DO NOT SHARE THIS:", private_key)
acct = Account.from_key(private_key)
print("Address:",acct.address)

SAVE BUT DO NOT SHARE THIS: 0x45d6df037f727330157610277ae46a81efafa524313b8d63bf5e116bfc70d556
Address: 0x47a7928bCb8024727F580607FA1A8068cf1F02A8
